# Project: Data Cleaning Challenge

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata


# display outputs inline
%matplotlib inline

# to see all columns and rows to using the display max column parameter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# set the display colwidth for seeing the entire text contents
pd.set_option('display.max_colwidth', -1)

C:\Users\USER\AppData\Local\Temp\ipykernel_42156\2121266886.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
# Importing data
fifa = pd.read_csv('fifa21 raw data v2.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_42156\3160915589.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv('fifa21 raw data v2.csv')


In [3]:
# make a copy of the data so as not to tamper with the original copy
fifa_clean = fifa.copy()

# Data Inspection

In [4]:
# view a sample of the data
fifa_clean.sample(5)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
4086,241501,A. Bareiro,Antonio Bareiro,https://cdn.sofifa.com/players/241/501/21_60.png,http://sofifa.com/player/241501/antonio-bareiro/210006/,Paraguay,31,71,71,\n\n\n\nClub Libertad,2014 ~ 2023,RM,167cm,68kg,Right,71,RM,"Jan 6, 2014",NaN,€1.6M,€800,€4.6M,286,63,65,36,63,59,313,71,52,59,67,64,399,84,81,80,69,85,271,47,49,90,41,44,278,55,40,66,77,40,54,96,22,39,35,55,14,13,6,13,9,1698,363,2 ★,3★,High,Medium,1 ★,82,56,66,70,33,56,1
10841,239873,A. Sobczyk,Alex Sobczyk,https://cdn.sofifa.com/players/239/873/21_60.png,http://sofifa.com/player/239873/alex-sobczyk/210006/,Austria,23,64,72,\n\n\n\nGórnik Zabrze,2020 ~ 2023,ST,185cm,76kg,Right,66,ST,"Aug 3, 2020",NaN,€1.3M,€2K,€1M,268,32,65,59,51,61,229,61,37,32,34,65,330,71,65,73,63,58,319,67,65,66,63,58,241,43,22,67,51,58,55,44,19,12,13,58,15,11,14,12,6,1489,319,3 ★,2★,Medium,Medium,1 ★,68,64,43,63,21,60,2
16306,253009,A. Cáceres,Ariel Cáceres,https://cdn.sofifa.com/players/253/009/21_60.png,http://sofifa.com/player/253009/ariel-caceres/210006/,Chile,20,58,73,\n\n\n\nUnión La Calera,2019 ~ 2024,RM,181cm,72kg,Right,59,RM,"May 25, 2019",NaN,€500K,€500,€543K,247,52,53,39,55,48,252,66,42,39,47,58,344,75,75,70,48,76,246,54,49,54,39,50,197,30,22,51,48,46,55,76,19,28,29,43,13,7,6,9,8,1405,307,3 ★,3★,Medium,Medium,1 ★,75,52,50,64,25,41,1
12870,242402,A. Eisa,Abo Eisa,https://cdn.sofifa.com/players/242/402/21_60.png,http://sofifa.com/player/242402/abo-eisa/210006/,Sudan,24,63,69,\n\n\n\nScunthorpe United,2019 ~ 2021,LM,180cm,75kg,Right,64,RM,"Aug 15, 2019",NaN,€750K,€4K,€975K,265,58,57,44,54,52,273,66,55,43,48,61,408,90,90,90,52,86,318,60,69,64,64,61,217,40,23,57,49,48,59,86,23,31,32,44,6,9,5,12,12,1611,354,3 ★,3★,High,Medium,1 ★,90,58,52,67,28,59,6
2825,236190,Roberto Soira,Roberto Júnior Soira Souza,https://cdn.sofifa.com/players/236/190/21_60.png,http://sofifa.com/player/236190/roberto-junior-soira-souza/210006/,Brazil,24,73,73,\n\n\n\nCoritiba,2019 ~ 2023,CB,187cm,83kg,Right,73,CB,"Jan 1, 2019",NaN,€3.1M,€15K,€7.4M,257,40,22,72,65,58,264,64,41,49,42,68,277,58,52,52,73,42,307,70,38,53,79,67,294,81,68,25,53,67,66,230,72,80,78,48,8,15,8,6,11,1677,360,2 ★,2★,Medium,Medium,1 ★,55,45,52,63,74,71,2


#### Observations
1. Name of clubs are messy, they contain irrelevant characters
2. The Contract column values dont follow the same format
3. The Height column values arent well formatted so would likely give rise to wrong datatype, this is observed in the weight column too.
4. The weight column is graduated in the wrong units, it ought to be in lbs
5. Loan end date looks to contain lots of NAN values.
6. The Value, Wage and Release Clause columns arent well formatted, the currency type should be in dollars and some conversions should be made.
7. The W/F, SM and IR columns are not well formatted and would likely give rise to the wrong datatype
8. Some Column Names arent descriptive enough.
9. There are inconsistencies in some values, like in Hits column where some data have a K in them.

In [5]:
# Checking for duplicates
fifa_clean[fifa_clean.duplicated()]

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits


#### No duplicate data in the dataset


In [6]:
# Identifying Null Values
null_counts = fifa_clean.isnull().sum()  # Count the number of null values in each column
null_cols = null_counts[null_counts > 0].index.tolist()  # Get the column names with null values

# Create a new dataframe with only the columns with null values and their corresponding null counts
null_fifa = pd.DataFrame({'Columns': null_cols, 'Null Count': null_counts[null_cols].tolist()})

null_fifa

,Columns,Null Count
0,Loan Date End,17966
1,Hits,2595


#### Observations
1. Only two columns contain Null Values, the Loan Date End and the Hits column, the Loan Date End Nulls seem to be significant.

In [7]:
# checking the datatypes and other information for each columns for each column
fifa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

#### Observation
1. There are some inappropriate data type assigned ro some columns

In [8]:
# view summary statistics of the fifa data
fifa_clean.describe()

,ID,Age,↓OVA,POT,BOV,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY
count,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.00000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000
mean,226403.384794,25.194109,65.718636,71.136414,66.751726,248.938142,49.688392,45.842405,51.942726,58.768112,42.696507,256.479214,55.587491,47.246957,42.391011,52.725381,58.528373,317.718689,64.357553,64.412930,63.366721,61.655619,63.925865,296.605195,57.801676,64.577375,62.653986,64.786922,46.785236,254.199747,55.591285,46.394489,50.298962,53.857105,48.057906,58.07366,139.909268,46.559197,47.705622,45.644449,81.436693,16.416302,16.207124,16.076453,16.217187,16.519627,1595.286949,355.702197,67.453975,53.457031,57.681016,62.875020,49.866221,64.368934
std,27141.054157,4.710520,6.968999,6.114635,6.747193,74.299428,18.131153,19.567081,17.294409,14.519106,17.646937,78.650601,18.761314,18.207790,17.227947,15.178151,16.565892,55.879046,14.889981,14.638743,14.596277,9.072114,14.062285,50.732390,13.308747,11.844870,15.804223,12.488672,19.300534,64.595613,17.140414,20.698078,19.428701,13.708574,15.655999,12.07911,61.219472,20.139324,21.367690,20.922087,84.665579,17.554194,16.816305,16.491103,17.002239,17.854079,269.874789,40.761117,10.677859,13.827425,10.081857,9.927415,16.443213,9.601883
min,41.000000,16.000000,47.000000,47.000000,48.000000,42.000000,6.000000,3.000000,5.000000,7.000000,3.000000,40.000000,5.000000,4.000000,5.000000,5.000000,5.000000,122.000000,13.000000,12.000000,14.000000,24.000000,12.000000,122.000000,18.000000,15.000000,12.000000,16.000000,4.000000,50.000000,9.000000,3.000000,2.000000,9.000000,6.000000,12.00000,20.000000,3.000000,5.000000,4.000000,10.000000,2.000000,2.000000,2.000000,2.000000,2.000000,747.000000,232.000000,25.000000,16.000000,25.000000,25.000000,12.000000,28.000000
25%,210135.000000,21.000000,61.000000,67.000000,62.000000,222.000000,38.000000,30.000000,44.000000,54.000000,30.000000,222.000000,49.000000,35.000000,31.000000,43.000000,54.000000,289.000000,57.000000,57.000000,55.000000,56.000000,56.000000,264.000000,48.000000,58.000000,55.000000,57.000000,32.000000,227.500000,44.000000,25.000000,40.000000,45.000000,39.000000,50.00000,83.000000,29.000000,27.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1452.000000,327.000000,61.000000,44.000000,51.000000,57.000000,35.000000,58.000000
50%,232418.000000,25.000000,66.000000,71.000000,67.000000,263.000000,54.000000,49.000000,55.000000,62.000000,44.000000,269.000000,61.000000,49.000000,41.000000,56.000000,63.000000,327.000000,67.000000,67.000000,66.000000,62.000000,66.000000,302.000000,59.000000,65.000000,66.000000,66.000000,51.000000,263.000000,58.000000,53.000000,55.000000,55.000000,49.000000,59.00000,159.000000,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1627.000000,356.000000,68.000000,56.000000,58.000000,64.000000,53.000000,65.000000
75%,246922.500000,29.000000,70.000000,75.000000,71.000000,297.

# Data Cleaning

### Handling inconsistencies in the data values

#### Rename Columns

In [9]:
fifa_clean.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [10]:
# Give columns more descriptive names for easy understanding, Abbreviations can be ambiguous and can lead to misunderstandings when analyzing the data
fifa_clean.rename(columns = {'LongName':'Full Name','Age':'Age(2021)', '↓OVA':'Overall Rating', 'POT':'Potential Rating',
                             'Height':'Height(ft)', 'Weight':'Weight(lbs)', 'BOV':'Best Overall Rating',
                            'Value':'Value(€)', 'Wage':'Wage(€)', 'ReleaseClause':'Release Clause(€)',
                            'W/F':'Weaker Foot(★)', 'SM':'Skils(★)', 'A/W':'Attacking Contribution',
                             'D/W':'Defensive Contribution', 'IR':'Injury Resistance(★)', 'PAC':'Pace', 'SHO':'Shot', 
                             'PAS':'Pass Accuracy', 'DRI':'Dribbling Ability',
                            'DEF':'Defensive Ability', 'PHY': 'Physicality', 'Hits': 'Profile Views'}, inplace = True)

fifa_clean.columns

Index(['ID', 'Name', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality',
       'Age(2021)', 'Overall Rating', 'Potential Rating', 'Club', 'Contract',
       'Positions', 'Height(ft)', 'Weight(lbs)', 'Preferred Foot',
       'Best Overall Rating', 'Best Position', 'Joined', 'Loan Date End',
       'Value(€)', 'Wage(€)', 'Release Clause', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Base Stats', 'Weaker

#### Datatype Conversion

In [11]:
fifa_clean['ID'] = fifa_clean['ID'].astype(str)
fifa_clean['Preferred Foot'] = fifa_clean['Preferred Foot'].astype('category')
fifa_clean['Joined'] = pd.to_datetime(fifa_clean['Joined'])
fifa_clean['Loan Date End'] = pd.to_datetime(fifa_clean['Loan Date End'])
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].astype('category')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].astype('category')
fifa_clean['Attacking Contribution'] = fifa_clean['Attacking Contribution'].astype('category')
fifa_clean['Defensive Contribution'] = fifa_clean['Defensive Contribution'].astype('category')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].astype('category')

#### Cleaning the Profile Views Column

This column has 2595 missing  values, values of over 1000 are represented with a decimal and a k, handling these issue would make the column cleaner, To fill the missing values, the mode would be used as the data is skewed

In [12]:
# looking at the Profile Views column to observe inconsistencies
fifa_clean['Profile Views'].loc[6:8]

6    246 
7    120 
8    1.6K
Name: Profile Views, dtype: object

In [13]:
# Replace 'K' and 'M' with their corresponding numeric values in the 'Profile Views' column
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].str.replace('K', 'e3').str.replace('M', 'e6')

# Convert the 'Profile Views' column to numeric type
fifa_clean['Profile Views'] = pd.to_numeric(fifa_clean['Profile Views'])

# Filter rows where the 'Profile Views' column contains 'K'
k_values = fifa_clean[fifa_clean['Profile Views'].astype(str).str.contains('e3')]

fifa_clean['Profile Views'].loc[6:8]

6    246.0 
7    120.0 
8    1600.0
Name: Profile Views, dtype: float64

In [14]:
# check the skewness of the hits values to determine what measure of center is best to fill null values
fifa_clean['Profile Views'].skew()

31.04253072771258

In [15]:
# get the mode to fill in na with
mode_hits = fifa_clean['Profile Views'].mode()
mode_hits

0    1.0
Name: Profile Views, dtype: float64

In [16]:
# fill in missing values in Profile Views column
fifa_clean['Profile Views'].fillna(1.0, inplace=True)

# change profile views to int data type
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].astype(int)

# Check to confirm there are no Null values
fifa_clean[fifa_clean['Profile Views'].isna()]

,ID,Name,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Contract,Positions,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Loan Date End,Value(€),Wage(€),Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views


####  Cleaning the Contract column

The Contract Column is having many discrepancies in value format, it also has some relationship with the Joined and loan date end column, from the contract column, a contract type column would be derived, this column tells us if the players contract is a permanent deal, a loan deal or a free agent(not under contract with any football club and is therefore free to sign with any club of their choosing). After which a contract start and contract end column is derived from the contract column, joined column and loan date column, these new columns tells when a player contract started, for a free agent this would be non applicable (N/A) as he has no contract while the contract end tells when the contracts would be terminated, for free agents the contract end would be N/A (not applicable) as a player without a contract cant have end date

In [17]:
fifa_clean['Contract'].loc[397:405]

397    Free                
398    2019 ~ 2023         
399    2017 ~ 2023         
400    2019 ~ 2024         
401    2014 ~ 2026         
402    2018 ~ 2023         
403    2019 ~ 2023         
404    2019 ~ 2024         
405    Jun 30, 2021 On Loan
Name: Contract, dtype: object

In [18]:
fifa_clean['Contract Type'] = fifa_clean['Contract'].apply(lambda x: 'Free' if x == 'Free' 
                                         else 'On Loan' if 'On Loan' in x 
                                         else 'Permanent' if '~' in x 
                                         else np.nan)


In [19]:
# define a function to parse the values in the 'Joined' and 'Loan Date End' columns
def parse_dates(row):
    if row['Contract'] == 'Free':
        start = np.nan
        end = np.nan
    elif 'On Loan' in row['Contract']:
        start = pd.to_datetime(row['Joined']).year
        end = row['Contract'].split(' ')[2]
    else:
        start = int(row['Contract'][:4])
        end = int(row['Contract'][-4:])
    return pd.Series({'Contract Start': start, 'Contract End': end})

# apply the function to create the new columns
fifa_clean[['Contract Start', 'Contract End']] = fifa_clean.apply(parse_dates, axis=1)


# convert float columns to object datatype and remove the unwanted characters
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype(str)
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype(str)

# Define a function to remove the '.i0' part from a string
def remove_dot_i(s):
    return s.split('.')[0]

# Apply the function to the fifa_clean['Contract Start']  and fifa_clean['Contract end'] column using .map()
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].map(remove_dot_i)
fifa_clean['Contract End'] = fifa_clean['Contract End'].map(remove_dot_i)

# Assigning the contract start and contract end columns into categories
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype('category')
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype('category')

# view samples of the result
fifa_clean[['Contract Type', 'Contract Start', 'Contract End']].loc[397:405]

,Contract Type,Contract Start,Contract End
397,Free,nan,nan
398,Permanent,2019,2023
399,Permanent,2017,2023
400,Permanent,2019,2024
401,Permanent,2014,2026
402,Permanent,2018,2023
403,Permanent,2019,2023
404,Permanent,2019,2024
405,On Loan,2017,2021


#### Cleaning the values in the Club, Name and Full Name columns

The club column values have messy and irrelevant characters in them, this would be cleaned. The Name, Full Name and Club colums have some diacritic characters whic could arise due to different languages and would affect alphabetic ordering during visualizations

In [20]:
# take a look at the column to see the format of naming
fifa_clean['Club'].sample(3)

7836     \n\n\n\nBoavista FC     
3381     \n\n\n\nSheffield United
13356    \n\n\n\nEl Nacional     
Name: Club, dtype: object

In [21]:
# Replacing the '\n' with an empty string
fifa_clean['Club'] = fifa_clean['Club'].str.replace('\n', '')
fifa_clean['Club'].sample(3)

8225     Newell's Old Boys
5334     Empoli           
13117    TSV Hartberg     
Name: Club, dtype: object

In [22]:
# Examples of diacritic names
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Süle          
Full Name    Niklas Süle      
Club         FC Bayern München
Name: 110, dtype: object

In [23]:
# Define regular expression pattern to match special characters in Name, LongName and Club columns
def remove_diacritics(text):
    """
    This function removes diacritics (accent marks) from text
    """
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

# Apply remove_diacritics function to Name, LongName, and Club columns
fifa_clean['Name'] = fifa_clean['Name'].apply(remove_diacritics)
fifa_clean['Full Name'] = fifa_clean['Full Name'].apply(remove_diacritics)
fifa_clean['Club'] = fifa_clean['Club'].apply(remove_diacritics)

# Check if Diacritic Names were removed
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Sule          
Full Name    Niklas Sule      
Club         FC Bayern Munchen
Name: 110, dtype: object

### 

####  Cleaning the Height column
The height column has values with contrasting units, some are in cm and others in feet, to clean it the units are removed from the values and then they are all converted to ftin in decimals, a descriptive column name would ensure the values are easily understood. The ftin unit is a more appropriate unit here because the weight column is tobe formatted in lbs, with this if need be for creating a BMI column it wont need any more formatting

In [24]:
fifa_clean['Height(ft)'].sample(6)

12042    183cm
5115     188cm
17176    184cm
4515     176cm
9199     170cm
18651    190cm
Name: Height(ft), dtype: object

In [25]:
# define a function that converts cm to feets and inches, This is my preferred unit as the weight is in lbs, this makes calculations that demands precision like the BMI easy

def convert_cm_to_feet(cm):
    cm = float(cm.replace('cm', '')) # remove 'cm' units and convert to float
    feet = int(cm / 30.48) # calculate number of feet
    inches = round((cm / 2.54) % 12) # calculate number of inches and round to nearest integer
    return f"{feet}'{inches}\"" # return result in feet and inches format

# apply the conversion function only to values that are in cm
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(lambda x: convert_cm_to_feet(x) if 'cm' in x else x)

# view the result
fifa_clean['Height(ft)'].sample(5)

5667    5'10"
8400    6'1" 
6940    5'6" 
4324    5'11"
987     6'0" 
Name: Height(ft), dtype: object

In [26]:
# define a function to convert feet and inches format to decimal format
def convert_feet_to_dec(feet):
    feet_list = feet.split("'")
    ft = float(feet_list[0])
    if len(feet_list) > 1:
        inches = float(feet_list[1].replace('"', ''))
    else:
        inches = 0
    dec = ft + inches / 12
    return round(dec, 2) # round to 1 decimal place


# apply the conversion function to the 'Height' column
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(convert_feet_to_dec)

# view result
fifa_clean['Height(ft)'].sample(6)

18800    6.00
12692    5.75
19       5.50
17310    6.17
13897    5.67
6808     6.33
Name: Height(ft), dtype: float64

#### Cleaning the Weight Column
The weight column has values with contrasting units, some are in lbs and others in kg, to clean it the units are removed from the values and then they are all converted to lbs, a descriptive column name would ensure the values are easily understood. The lbs unit is a more appropriate unit here because the height column is formatted in ftin, with this if need be for creating a BMI column it wont need any more formatting

In [27]:
fifa_clean['Weight(lbs)'].sample(6)

16718    72kg
15327    73kg
768      64kg
4969     77kg
17001    69kg
3274     60kg
Name: Weight(lbs), dtype: object

In [28]:
# define a function that converts kg to lbs
def convert_kg_to_lbs(kg):
    kg = float(kg.replace('kg', '')) # remove 'kg' units and convert to float
    lbs = int(kg * 2.20462) # calculate kg
    return f"{lbs}lbs" #returns result in lbs format

# apply the conversion function only to values that are in kg
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].apply(lambda x: convert_kg_to_lbs(x) if 'kg' in x else x)

# remove the lbs units from the vale
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].str.replace('lbs', '')

# convert to float datatype
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].astype(float)

# view result of code
fifa_clean['Weight(lbs)'].sample(3)

580      176.0
16450    158.0
1856     171.0
Name: Weight(lbs), dtype: float64

#### Cleaning the Value, Wage and Release Clause columns
The Value, Wage and Release columns have similar mess issues, have currency symbols in them and are formatted in thousand and millions decimal points and have K and M attached to them. These symbols would be dropped and the thousands and millions format would be made uniform. The choice of of the euro(€) currency is because most transfer deals are negotiated in euros

In [29]:
fifa_clean[['Value(€)','Wage(€)', 'Release Clause']].head()

,Value(€),Wage(€),Release Clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [30]:
# Define a function to convert euro and k/m suffixes to USD and millions
def convert_value(val):
    if val.startswith('€'):
        val = val.replace('€', '')
        if val.endswith('K'):
            val = float(val.replace('K', '')) * 1000
        elif val.endswith('M'):
            val = float(val.replace('M', '')) * 1000000
    
    return pd.to_numeric(val)

# Apply the conversion function to the DataFrame
fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']] = fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']].applymap(convert_value)

# Set the display format of floats to include a comma delimiter
pd.options.display.float_format = '{:,.2f}'.format

fifa_clean.rename(columns={'Release Clause': 'Release Clause(€)'}, inplace=True)

# quick glance at the resulting data
fifa_clean[['Value(€)','Wage(€)', 'Release Clause(€)']].sample(5)

,Value(€),Wage(€),Release Clause(€)
14986,"275,000.00","2,000.00","408,000.00"
2086,"9,500,000.00","34,000.00","15,300,000.00"
18224,"190,000.00","3,000.00",0.00
16634,"575,000.00","8,000.00",0.00
13775,"850,000.00",700.00,"701,000.00"


#### Cleaning columns with star ratings (Weaker Foot, Skills, Injury Resistance)
The columns with values as star ratings have symbols to represent the number of stars received for each category by a player, these star symbols would be dropped

In [31]:
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].head()

,Weaker Foot(★),Skils(★),Injury Resistance(★)
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [32]:
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].str.replace('\u2605', '')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].str.replace('\u2605', '')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].str.replace('\u2605', '')

# quick glance at the resulting data
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].sample(5)

,Weaker Foot(★),Skils(★),Injury Resistance(★)
16311,2,3,1
7580,5,2,1
14678,2,2,1
5664,2,3,1
10549,3,3,1


#### Current ages of players from the Age column 
The age column houses ages of players in 2021 when this data was collated to have an updated age value for the year of 2023, a new column would be derived from the previous age column

In [33]:
# add 2 to their previous age to get their current age
fifa_clean['Age(2023)'] = fifa_clean['Age(2021)'] + 2

# a quick glance of samples of the data
fifa_clean[['Age(2021)', 'Age(2023)']].sample(5)

,Age(2021),Age(2023)
4765,26,28
10785,25,27
1659,24,26
8546,22,24
17950,23,25


#### Drop unnecessary columns
The contract and loan date end columns have been formatted already to give newer more imformative and cleaner columns(Contract Type, Contract Start and Contract End) so should be drop, the positions column looks messy and a more unique column(Best Positions) is available to tell us what position the player plays at

In [34]:
# drop the contract, positions and loan date end columns
fifa_clean.drop(['Contract', 'Positions', 'Name', 'Loan Date End'], axis=1, inplace=True)
fifa_clean.columns

Index(['ID', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality', 'Age(2021)',
       'Overall Rating', 'Potential Rating', 'Club', 'Height(ft)',
       'Weight(lbs)', 'Preferred Foot', 'Best Overall Rating', 'Best Position',
       'Joined', 'Value(€)', 'Wage(€)', 'Release Clause(€)', 'Attacking',
       'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina',
       'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending',
       'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Total Stats', 'Base Stats', 'Weaker Foot(★)',
       'Skils(★)', 'Attacking Contribution'

#### The total stats column is an aggregate of the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping Column, each of these columns are also aggregates of other columns which are broken down below:
1.  Attacking Column = Crossing + Finishing + HEading Accuracy + Short Passing + Volleys
2. Skill Column = Dribbling + Curve + FK Accuracy + Long Passing + Ball Control 
3. Movement Column = Acceleration + Sprint Speed + Agility + Reactions + Balance 
4. Power Column = Shot Power + Jumping + Stamina + Strength + Long Shots
5. Mentality Column = Aggression + Interceptions + Positioning + Vision + Penalties
6. Defending Column = Marking + Standing Tackle + Sliding Tackle
7. Goalkeeping = GK Diving + GK Handling + GK Kicking + GK Positioning + GK Reflexes
#### This leaves out the composure column, a quick visit to the sofifa.com site shows this is a little oversight and the composure column was supposed to be added to the Mentality Column and the total affects the total stats.

Other attributes column would be accessed to be sure the calculations done were correct and then the composure column values would be inputted where they ought to bee used for calculations

##### Confirm Attacking column Calculations

In [35]:
# Create a new column "Total Attacking" to store the sum of the attacking attributes
fifa_clean["Total Attacking"] = fifa_clean[["Crossing", "Finishing", "Heading Accuracy", "Short Passing", "Volleys"]].sum(axis=1)

# Check if the "Total Attacking" matches the "Attacking" column for each row
fifa_clean["Attacking Check"] = fifa_clean["Attacking"] == fifa_clean["Total Attacking"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Attacking Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Attacking,Attacking Check


##### drop columns used in comparison

In [36]:
# Drop the "Total Attacking" and "Attacking Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Attacking", "Attacking Check"])

# a quick glance to confirm thy were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
3541,221323,Jerome Onguene,https://cdn.sofifa.com/players/221/323/21_60.png,http://sofifa.com/player/221323/jerome-onguene/210006/,Cameroon,22,72,80,FC Red Bull Salzburg,6.08,171.00,Right,74,CB,2018-07-01,"4,900,000.00","15,000.00","7,100,000.00",217,32,24,76,58,27,208,53,28,33,36,58,348,68,76,75,68,61,304,45,89,72,74,24,248,72,71,26,32,47,58,217,73,73,71,59,8,14,12,13,12,1601,349,3,2,Medium,Medium,1,72,30,42,58,73,74,42,Permanent,2018,2022,24


##### Confirm Skill column Calculations

In [37]:
# Create a new column "Total Skill" to store the sum of the skill attributes
fifa_clean["Total Skill"] = fifa_clean[['Dribbling','Curve', 'FK Accuracy', 'Long Passing', 'Ball Control']].sum(axis=1)

# Check if the "Total Skill" matches the "Skill" column for each row
fifa_clean["Skill Check"] = fifa_clean["Skill"] == fifa_clean["Total Skill"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Skill Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Skill,Skill Check


##### drop columns used in comparison

In [38]:
# Drop the "Total Skill" and "Skill Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Skill", "Skill Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
12525,252044,Leonel Picco,https://cdn.sofifa.com/players/252/044/21_60.png,http://sofifa.com/player/252044/leonel-picco/210006/,Argentina,20,63,73,Arsenal de Sarandi,6.17,174.00,Right,64,CDM,2019-01-01,"1,000,000.00","2,000.00","908,000.00",243,41,37,55,71,39,263,61,43,33,61,65,265,53,55,53,55,49,309,61,59,71,73,45,265,71,51,45,51,47,49,185,61,63,61,52,5,13,6,14,14,1582,345,3,2,Medium,Medium,1,54,44,56,61,59,71,3,Permanent,2019,2022,22


##### Confirm Movement column Calculations

In [39]:
# Create a new column "Total Movement" to store the sum of the Movement attributes
fifa_clean["Total Movement"] = fifa_clean[['Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance']].sum(axis=1)

# Check if the "Total Movement" matches the "Movement" column for each row
fifa_clean["Movement Check"] = fifa_clean["Movement"] == fifa_clean["Total Movement"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Movement Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Movement,Movement Check


##### drop columns used in comparison

In [40]:
# Drop the "Total Movement" and "Movement Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Movement", "Movement Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
5424,243716,Silvester van der Water,https://cdn.sofifa.com/players/243/716/21_60.png,http://sofifa.com/player/243716/silvester-van-der-water/210006/,Netherlands,23,69,73,Heracles Almelo,6.00,163.00,Left,70,LM,2018-07-01,"2,000,000.00","4,000.00","2,100,000.00",306,64,70,50,64,58,306,74,67,43,54,68,368,78,84,77,66,63,310,72,47,67,60,64,280,68,24,66,55,67,64,129,32,52,45,47,8,9,10,14,6,1746,384,3,3,High,High,1,81,68,60,72,40,63,3,Permanent,2018,2021,25


##### Confirm Power column Calculations

In [41]:
# Create a new column "Total Power" to store the sum of the Power attributes
fifa_clean["Total Power"] = fifa_clean[['Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots']].sum(axis=1)

# Check if the "Total Power" matches the "Power" column for each row
fifa_clean["Power Check"] = fifa_clean["Power"] == fifa_clean["Total Power"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Power Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Power,Power Check


##### drop columns used in comparison

In [42]:
# Drop the "Total Power" and "Power Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Power", "Power Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
4538,213512,Guðmundur Þorarinsson,https://cdn.sofifa.com/players/213/512/21_60.png,http://sofifa.com/player/213512/gudhmundur-thorarinsson/210006/,Iceland,28,70,70,New York City FC,6.00,165.00,Left,70,LWB,2020-01-28,"1,400,000.00","5,000.00","2,300,000.00",291,78,56,36,66,55,353,69,73,73,67,71,340,69,57,73,67,74,353,75,66,79,64,69,329,76,65,53,72,63,67,190,61,64,65,56,16,13,10,10,7,1912,396,4,4,Medium,Medium,1,62,63,70,70,61,70,5,Permanent,2020,2020,30


##### Update Mentality column by adding Composure values to its attributes

In [43]:
# suming  up the  'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties'and add the 'Composure' column to make the Mentality column
fifa_clean['Mentality'] = fifa_clean[[ 'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties', 'Composure']].sum(axis=1)

# a quick glance to confirm they were dropped
fifa_clean[['Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure']].sample(4)

,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure
3501,415,75,69,59,74,69,69
372,197,26,21,11,60,15,64
13974,324,62,16,65,66,55,60
28,471,73,49,89,85,86,89


##### Confirm Defending column Calculations

In [44]:
# Create a new column "Total Defending" to store the sum of the defending attributes
fifa_clean["Total Defending"] = fifa_clean[['Marking', 'Standing Tackle','Sliding Tackle']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Defending Check"] = fifa_clean["Defending"] == fifa_clean["Total Defending"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Defending Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Defending,Defending Check


##### drop columns used in comparison

In [45]:
# Drop the "Total Defending" and "Defending Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Defending", "Defending Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
11744,233173,Hokuto Shimoda,https://cdn.sofifa.com/players/233/173/21_60.png,http://sofifa.com/player/233173/hokuto-shimoda/210006/,Japan,28,64,64,Kawasaki Frontale,5.58,145.00,Left,65,LM,2018-01-06,"575,000.00","3,000.00","585,000.00",268,63,52,42,67,44,299,58,52,56,65,68,373,80,77,77,62,77,340,71,77,85,53,54,332,56,57,52,66,44,57,156,49,56,51,49,10,13,11,9,6,1760,376,4,2,Medium,Medium,1,78,55,64,64,52,63,2,Permanent,2018,2021,30


##### Confirm Goalkeeping column Calculations

In [46]:
# Create a new column "Total GK" to store the sum of the Goalkeeping attributes
fifa_clean["Total GK"] = fifa_clean[['GK Diving', 'GK Handling','GK Kicking', 'GK Positioning', 'GK Reflexes']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["GK Check"] = fifa_clean["Goalkeeping"] == fifa_clean["Total GK"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["GK Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total GK,GK Check


##### drop columns used in comparison

In [47]:
# Drop the "Total GK" and "GK Check" columns
fifa_clean = fifa_clean.drop(columns=["Total GK", "GK Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
11899,184285,Christian Schneuwly,https://cdn.sofifa.com/players/184/285/21_60.png,http://sofifa.com/player/184285/christian-schneuwly/210006/,Switzerland,32,64,64,FC Lausanne-Sport,5.83,163.00,Right,65,RM,2019-08-08,"450,000.00","5,000.00","508,000.00",309,65,63,52,68,61,315,66,59,59,65,66,329,66,68,65,59,71,335,70,67,70,62,66,352,50,58,58,63,58,65,166,57,54,55,39,8,6,11,7,7,1780,380,3,3,Medium,Medium,1,67,64,65,66,56,62,2,Permanent,2019,2021,34


#### Total stats represent the final value of all the stats a character or object has, which includes their base stats, any bonuses or modifications gained through equipment or leveling up, and any other factors that affect their abilities. Therefore, adding the base stats to the total stats would result in double-counting the base stats. So base stats would be left out in this aggregation

##### Update Total Stats column by with updating values of its attributes

In [48]:
# suming  up the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping to make the Mentality column
fifa_clean['Total Stats'] = fifa_clean[[ 'Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sum(axis=1)

# a view of the resulting total stats column
fifa_clean[[ 'Total Stats','Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sample()

,Total Stats,Attacking,Skill,Movement,Power,Mentality,Defending
9364,778,86,81,210,196,161,44


#### Confirm Base Stats column Calculations
The Base stats refer to the six basic categories of skills used to rate a player's abilities in FIFA. These stats are typically used as a starting point for evaluating a player's performance, and they provide a basic overview of a player's abilities. The six columns would be added to compare if they sum up to the base stats

##### Confirm Base Stats column Calculations

In [49]:
# Create a new column "Total Base Stats" to store the sum of the Base Stats attributes
fifa_clean["Total Base Stats"] = fifa_clean[['Pace', 'Shot',
       'Pass Accuracy', 'Dribbling Ability', 'Defensive Ability',
       'Physicality']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Base Stats Check"] = fifa_clean["Base Stats"] == fifa_clean["Total Base Stats"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Base Stats Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Base Stats,Base Stats Check


##### drop columns used in comparison

In [50]:
# Drop the "Total Base Stats" and "Base Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Base Stats", "Base Stats Check"])

# a view of a sample of the data
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
3048,200911,Bogdan Stancu,https://cdn.sofifa.com/players/200/911/21_60.png,http://sofifa.com/player/200911/bogdan-stancu/210006/,Romania,33,73,73,Genclerbirligi SK,6.00,165.00,Right,73,ST,2019-01-28,"1,900,000.00","17,000.00","5,000,000.00",347,62,75,70,69,71,329,70,58,61,67,73,351,71,73,74,71,62,349,72,73,65,70,69,402,61,22,82,78,80,79,97,22,45,30,61,7,11,14,14,15,1875,387,3,3,High,High,1,72,74,68,71,35,67,6,Permanent,2019,2021,35


#### Reindex Columns

Changing the order of columns can be helpful for organizing the data in a way that makes it easier to understand or analyze.

In [51]:
fifa_clean.insert(loc=7, column='Age(2023)', value=fifa_clean.pop('Age(2023)'))
fifa_clean.insert(loc=8, column='Contract Type', value=fifa_clean.pop('Contract Type'))
fifa_clean.insert(loc=9, column='Contract Start', value=fifa_clean.pop('Contract Start'))
fifa_clean.insert(loc=10, column='Contract End', value=fifa_clean.pop('Contract End'))
fifa_clean.insert(loc=66, column='Pace', value=fifa_clean.pop('Pace'))
fifa_clean.insert(loc=67, column='Shot', value=fifa_clean.pop('Shot'))
fifa_clean.insert(loc=68, column='Pass Accuracy', value=fifa_clean.pop('Pass Accuracy'))
fifa_clean.insert(loc=69, column='Dribbling Ability', value=fifa_clean.pop('Dribbling Ability'))
fifa_clean.insert(loc=70, column='Defensive Ability', value=fifa_clean.pop('Defensive Ability'))
fifa_clean.insert(loc=71, column='Physicality', value=fifa_clean.pop('Physicality'))


fifa_clean.sample(3)

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Age(2023),Contract Type,Contract Start,Contract End,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Profile Views
15831,213333,Osama Ashoor,https://cdn.sofifa.com/players/213/333/21_60.png,http://sofifa.com/player/213333/osama-ashoor/210006/,Saudi Arabia,30,59,32,Permanent,2019,2025,59,Abha Club,5.58,165.00,Right,59,RB,2019-07-01,"190,000.00","2,000.00","198,000.00",200,54,26,43,48,29,208,51,35,34,46,42,330,68,70,64,53,75,247,24,61,70,69,23,273,53,58,48,37,33,44,173,59,56,58,50,7,15,10,10,8,1431,314,4,69,27,45,51,56,66,2,Medium,Medium,1,1
283,207877,Josef Martinez,https://cdn.sofifa.com/players/207/877/21_60.png,http://sofifa.com/player/207877/josef-martinez/210006/,Venezuela,27,81,29,Permanent,2017,2023,82,Atlanta United,5.58,152.00,Right,82,ST,2017-03-22,"30,000,000.00","14,000.00","33,100,000.00",359,53,85,77,69,75,335,76,68,57,55,79,441,87,90,91,81,92,390,84,90,72,78,66,383,66,20,85,58,79,75,55,20,20,15,60,12,14,14,12,8,1963,409,4,89,80,61,79,25,75,3,High,Medium,1,178
13098,231446,Marcus McGuane,https://cdn.sofifa.com/players/231/446/21_60.png,http://sofifa.com/player/231446/marcus-mcguane/210006/,England,21,62,23,On Loan,2020,2021,74,Oxford United,5.83,160.00,Right,64,CM,2020-02-27,"975,000.00","6,000.00",0.00,251,53,42,40,66,50,284,60,48,46,66,64,328,70,67,60,65,66,290,58,58,63,68,43,335,54,55,48,66,52,60,170,55,57,58,62,9,14,12,13,14,1658,356,3,68,47,62,62,54,63,3,Medium,Medium,1,14


In [52]:
# save cleaned fifa data
fifa_clean.to_csv('Fifa_2021_cleaned.csv',index=False)

# Print a message to confirm the file has been saved
print('fifa_clean.csv saved successfully.')

fifa_clean.csv saved successfully.
